# Preprocessamento:

+ Criar variável que identifique o anotador

+ Mover tags que começam com token ' ' (vazio)

+ Remover linhas com '\n' seguidos

+ REGEX:
    + Garantir letra e números onde tamanho for maior que 1.
    + Passar múltiplos símbolos para outra linha. Exemplo:  §3º -->  § \n 3 \n º
    + Remover pontuação de centenas dos números. Exemplo: 12.200 --> 12200

+ Visualização das sentenças com displacy (from spacy import displacy) 

+ Incluir POS tagging.

+ Tranformar o dado $x_i$ em um $x'_i$ que incorpora os 2 últimos e próximos tokens.

# Classificador

+ Visualização: Separar o conjunto de test em 2 ou 3 arquivos e visualizar o que o modelo classificou e o que os anotadores classificaram (separar por id do anotador).

+ Parâmetros utilizados no classificador.

+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

+ Para criar um contexto no erro imprimir 10 palavras antes e depois de dois erros.


# Instruções para os anotadores:

+ Atentar à marcação de tags que envolve espaço 

+ Atentar para não incluir espaço no início da Tag

# Organização do diretório: 

+ Manter toda a análise em somente um diretório

+ Formato do diretório com os datasets: /resources/dataset/

+ Notebooks:
    + '01 - Processamento.ipynb'
        + Gerar 'treino.csv' e teste.csv' para processamento
    + '02 - [CRF].ipynb' - Criando o modelo
        + Usar arquivo dos dados preprocessados gerado pelo notebook 1.
        + Gerar modelo (xxx.model)
    + '03 - Metricas.ipynb'

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import re 
from collections import Counter

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
import time
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [2]:
# Encontra todos os csv's dentro das pastas de 'mock'

extension = 'csv'
all_filenames = [i for i in glob.glob('mock/*/**/***/****.{}'.format(extension))]

In [3]:
all_filenames[:3]

['mock/161704902/[PRATICA_ETAPA_1]/Documentos/20180510_Rcl_22328_314302526.ner.csv',
 'mock/161704902/[PRATICA_ETAPA_1]/Documentos/20170904_ARE_1062176_312610974.ner.csv',
 'mock/161704902/[PRATICA_ETAPA_1]/Documentos/20161121_ARE_1009041_310751428.ner.csv']

# Preprocessamento 

In [4]:
# Cria uma tag de inicio e fim de arquivo em cada 'csv' antes de apendar todos eles.

frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ'] # Altera a primeira e ultima Tag desse csv
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
combined_csv = pd.concat(frames).reset_index(drop=True)
combined_csv.to_csv("combined_csv.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
combined_csv['Token'] = combined_csv['Token'].astype('str')

In [5]:
combined_csv.head(15), combined_csv.tail(20)

(         Token         Tag
 0       Ementa  INICIO_ARQ
 1                        O
 2            e           O
 3                        O
 4      Acórdão           O
 5                        O
 6   06/03/2018           O
 7                        O
 8     PRIMEIRA           O
 9                        O
 10       TURMA           O
 11                       O
 12          \n           O
 13          \n           O
 14  RECLAMAÇÃO           O,                               Token      Tag
 967890                           de        O
 967891                                     O
 967892                     Oliveira        O
 967893                                     O
 967894                       Duarte        O
 967895                                     O
 967896                           \n        O
 967897                           \n        O
 967898                    Assessora        O
 967899                            -        O
 967900                        Chefe        O


In [6]:
print("Número de linhas dos arquivos concatenados:", len(combined_csv['Tag']))

Número de linhas dos arquivos concatenados: 967910


In [7]:
combined_csv[-10:] # Conferindo se o index foi resetado

,Token,Tag
967900,Chefe,O
967901,,O
967902,do,O
967903,,O
967904,Plenário,O
967905,\n,O
967906,id,O
967907,:,O
967908,,O
967909,20141203_ADI_4350_285683668,FIM_ARQ


# Encontra parágrafo duplo no arquivo. Uma opção de separar por sentenças.

In [8]:
a_df = combined_csv #Simplifica o nome do arquivo para a função nao ficar grande demais.
starts = a_df[a_df['Token']=='\n'].index & a_df[a_df['Token'].shift(-1)=='\n'].index #Identifica os paragrafos duplos
print(u'Padrões(sentenças) encontrados:', len(starts))

Padrões(sentenças) encontrados: 9708


In [9]:
%%time

combined_csv['Sentence #'] = 'Sentence'

combined_csv['Sentence #'][:starts[0]+2] = 'Sentence %d'%(1) # Primeira sentença
combined_csv['Sentence #'][starts[-1]+2:] = 'Sentence %d'%(len(starts)+1) # Última sentença

for i in range(1,len(starts)):
    combined_csv['Sentence #'][starts[i-1]+2:starts[i]+2] = 'Sentence %d'%(i+1) 

combined_csv.head(), combined_csv.tail()

CPU times: user 5min 48s, sys: 271 ms, total: 5min 49s
Wall time: 5min 50s


(     Token         Tag  Sentence #
 0   Ementa  INICIO_ARQ  Sentence 1
 1                    O  Sentence 1
 2        e           O  Sentence 1
 3                    O  Sentence 1
 4  Acórdão           O  Sentence 1,
                               Token      Tag     Sentence #
 967905                           \n        O  Sentence 9709
 967906                           id        O  Sentence 9709
 967907                            :        O  Sentence 9709
 967908                                     O  Sentence 9709
 967909  20141203_ADI_4350_285683668  FIM_ARQ  Sentence 9709)

In [10]:
# Número de sentenças
len(combined_csv['Sentence #'].unique())

9709

# Atualiza a Tag que termina com 'Doutrinador' para 'Doutrina'

In [11]:
combined_csv.Tag.unique()

array(['INICIO_ARQ', 'O', 'B_Pessoa', 'I_Pessoa', 'B_Precedente',
       'I_Precedente', 'B_Ref. Legislativa', 'I_Ref. Legislativa',
       'B_Doutrina', 'I_Doutrina', 'FIM_ARQ', 'B_Doutrinador',
       'I_Doutrinador'], dtype=object)

In [12]:
# Strip o final 'dor' de todo o DataFrame (formato extremo)
# combined_csv.Tag = combined_csv.Tag.str.rstrip('dor')

In [13]:
# indices de onde a tag ocorre
indx = combined_csv[combined_csv.Tag.str.endswith('Doutrinador')].index.values

### Rodar apenas uma vez

In [14]:
%%time
# Demorado e custoso
for i in range(len(indx)):
    combined_csv.Tag.iloc[indx[i]] = combined_csv.Tag.iloc[indx[i]].rstrip('dor')

CPU times: user 4min 21s, sys: 192 ms, total: 4min 21s
Wall time: 4min 22s


In [15]:
combined_csv.Tag.unique()

array(['INICIO_ARQ', 'O', 'B_Pessoa', 'I_Pessoa', 'B_Precedente',
       'I_Precedente', 'B_Ref. Legislativa', 'I_Ref. Legislativa',
       'B_Doutrina', 'I_Doutrina', 'FIM_ARQ'], dtype=object)

# Remove enter duplo depois de criar as sentenças

In [18]:
# Teste para ver os casoso onde ocorre enter duplo.
for i in range(len(starts)):
    print(combined_csv.iloc[starts[i]:starts[i]+2][:5])

   Token Tag  Sentence #
12    \n   O  Sentence 1
13    \n   O  Sentence 1
   Token Tag  Sentence #
24    \n   O  Sentence 2
25    \n   O  Sentence 2
   Token Tag  Sentence #
98    \n   O  Sentence 3
99    \n   O  Sentence 3
    Token Tag  Sentence #
114    \n   O  Sentence 4
115    \n   O  Sentence 4
    Token Tag  Sentence #
161    \n   O  Sentence 5
162    \n   O  Sentence 5
    Token       Tag  Sentence #
204    \n  I_Pessoa  Sentence 6
205    \n  I_Pessoa  Sentence 6
    Token       Tag  Sentence #
208    \n  I_Pessoa  Sentence 7
209    \n  I_Pessoa  Sentence 7
    Token Tag  Sentence #
212    \n   O  Sentence 8
213    \n   O  Sentence 8
    Token Tag  Sentence #
263    \n   O  Sentence 9
264    \n   O  Sentence 9
    Token Tag   Sentence #
327    \n   O  Sentence 10
328    \n   O  Sentence 10
    Token Tag   Sentence #
399    \n   O  Sentence 11
400    \n   O  Sentence 11
    Token Tag   Sentence #
467    \n   O  Sentence 12
468    \n   O  Sentence 12
    Token Tag   Sentence #
5

      Token Tag    Sentence #
14945    \n   O  Sentence 155
14946    \n   O  Sentence 155
      Token Tag    Sentence #
14968    \n   O  Sentence 156
14969    \n   O  Sentence 156
      Token Tag    Sentence #
14991    \n   O  Sentence 157
14992    \n   O  Sentence 157
      Token Tag    Sentence #
15040    \n   O  Sentence 158
15041    \n   O  Sentence 158
      Token Tag    Sentence #
15223    \n   O  Sentence 159
15224    \n   O  Sentence 159
      Token Tag    Sentence #
15381    \n   O  Sentence 160
15382    \n   O  Sentence 160
      Token Tag    Sentence #
15441    \n   O  Sentence 161
15442    \n   O  Sentence 161
      Token Tag    Sentence #
15476    \n   O  Sentence 162
15477    \n   O  Sentence 162
      Token       Tag    Sentence #
15573    \n  I_Pessoa  Sentence 163
15574    \n  I_Pessoa  Sentence 163
      Token Tag    Sentence #
15771    \n   O  Sentence 164
15772    \n   O  Sentence 164
      Token Tag    Sentence #
15845    \n   O  Sentence 165
15846    \n   O  Sente

      Token Tag    Sentence #
33643    \n   O  Sentence 310
33644    \n   O  Sentence 310
      Token Tag    Sentence #
33752    \n   O  Sentence 311
33753    \n   O  Sentence 311
      Token                 Tag    Sentence #
33779    \n  I_Ref. Legislativa  Sentence 312
33780    \n  I_Ref. Legislativa  Sentence 312
      Token Tag    Sentence #
33786    \n   O  Sentence 313
33787    \n   O  Sentence 313
      Token Tag    Sentence #
33857    \n   O  Sentence 314
33858    \n   O  Sentence 314
      Token Tag    Sentence #
33997    \n   O  Sentence 315
33998    \n   O  Sentence 315
      Token Tag    Sentence #
34090    \n   O  Sentence 316
34091    \n   O  Sentence 316
      Token Tag    Sentence #
34215    \n   O  Sentence 317
34216    \n   O  Sentence 317
      Token Tag    Sentence #
34272    \n   O  Sentence 318
34273    \n   O  Sentence 318
      Token Tag    Sentence #
34507    \n   O  Sentence 319
34508    \n   O  Sentence 319
      Token Tag    Sentence #
34679    \n   O  Sente

      Token Tag    Sentence #
50088    \n   O  Sentence 469
50089    \n   O  Sentence 469
      Token Tag    Sentence #
50181    \n   O  Sentence 470
50182    \n   O  Sentence 470
      Token Tag    Sentence #
50234    \n   O  Sentence 471
50235    \n   O  Sentence 471
      Token Tag    Sentence #
50272    \n   O  Sentence 472
50273    \n   O  Sentence 472
      Token Tag    Sentence #
50298    \n   O  Sentence 473
50299    \n   O  Sentence 473
      Token Tag    Sentence #
50446    \n   O  Sentence 474
50447    \n   O  Sentence 474
      Token Tag    Sentence #
50843    \n   O  Sentence 475
50844    \n   O  Sentence 475
      Token Tag    Sentence #
50987    \n   O  Sentence 476
50988    \n   O  Sentence 476
      Token Tag    Sentence #
51094    \n   O  Sentence 477
51095    \n   O  Sentence 477
      Token Tag    Sentence #
51214    \n   O  Sentence 478
51215    \n   O  Sentence 478
      Token Tag    Sentence #
51265    \n   O  Sentence 479
51266    \n   O  Sentence 479
      Toke

      Token Tag    Sentence #
63313    \n   O  Sentence 629
63314    \n   O  Sentence 629
      Token Tag    Sentence #
63345    \n   O  Sentence 630
63346    \n   O  Sentence 630
      Token Tag    Sentence #
63366    \n   O  Sentence 631
63367    \n   O  Sentence 631
      Token Tag    Sentence #
63490    \n   O  Sentence 632
63491    \n   O  Sentence 632
      Token Tag    Sentence #
63587    \n   O  Sentence 633
63588    \n   O  Sentence 633
      Token Tag    Sentence #
63678    \n   O  Sentence 634
63679    \n   O  Sentence 634
      Token Tag    Sentence #
63738    \n   O  Sentence 635
63739    \n   O  Sentence 635
      Token Tag    Sentence #
63754    \n   O  Sentence 636
63755    \n   O  Sentence 636
      Token Tag    Sentence #
63903    \n   O  Sentence 637
63904    \n   O  Sentence 637
      Token Tag    Sentence #
63912    \n   O  Sentence 638
63913    \n   O  Sentence 638
      Token Tag    Sentence #
63920    \n   O  Sentence 639
63921    \n   O  Sentence 639
      Toke

...

885101    \n   O  Sentence 8962
       Token Tag     Sentence #
885196    \n   O  Sentence 8963
885197    \n   O  Sentence 8963
       Token Tag     Sentence #
885280    \n   O  Sentence 8964
885281    \n   O  Sentence 8964
       Token Tag     Sentence #
885360    \n   O  Sentence 8965
885361    \n   O  Sentence 8965
       Token Tag     Sentence #
885400    \n   O  Sentence 8966
885401    \n   O  Sentence 8966
       Token Tag     Sentence #
885416    \n   O  Sentence 8967
885417    \n   O  Sentence 8967
       Token Tag     Sentence #
885521    \n   O  Sentence 8968
885522    \n   O  Sentence 8968
       Token Tag     Sentence #
885610    \n   O  Sentence 8969
885611    \n   O  Sentence 8969
       Token Tag     Sentence #
885820    \n   O  Sentence 8970
885821    \n   O  Sentence 8970
       Token Tag     Sentence #
885966    \n   O  Sentence 8971
885967    \n   O  Sentence 8971
       Token Tag     Sentence #
886083    \n   O  Sentence 8972
886084    \n   O  Sentence 8972
       T

899811    \n   O  Sentence 9112
       Token Tag     Sentence #
899986    \n   O  Sentence 9113
899987    \n   O  Sentence 9113
       Token Tag     Sentence #
900325    \n   O  Sentence 9114
900326    \n   O  Sentence 9114
       Token Tag     Sentence #
900456    \n   O  Sentence 9115
900457    \n   O  Sentence 9115
       Token Tag     Sentence #
900685    \n   O  Sentence 9116
900686    \n   O  Sentence 9116
       Token Tag     Sentence #
900963    \n   O  Sentence 9117
900964    \n   O  Sentence 9117
       Token Tag     Sentence #
901111    \n   O  Sentence 9118
901112    \n   O  Sentence 9118
       Token Tag     Sentence #
901202    \n   O  Sentence 9119
901203    \n   O  Sentence 9119
       Token Tag     Sentence #
901272    \n   O  Sentence 9120
901273    \n   O  Sentence 9120
       Token Tag     Sentence #
901339    \n   O  Sentence 9121
901340    \n   O  Sentence 9121
       Token Tag     Sentence #
901405    \n   O  Sentence 9122
901406    \n   O  Sentence 9122
       T

       Token Tag     Sentence #
917319    \n   O  Sentence 9271
917320    \n   O  Sentence 9271
       Token Tag     Sentence #
917477    \n   O  Sentence 9272
917478    \n   O  Sentence 9272
       Token Tag     Sentence #
917523    \n   O  Sentence 9273
917524    \n   O  Sentence 9273
       Token           Tag     Sentence #
918347    \n  I_Precedente  Sentence 9274
918348    \n  I_Precedente  Sentence 9274
       Token Tag     Sentence #
918369    \n   O  Sentence 9275
918370    \n   O  Sentence 9275
       Token Tag     Sentence #
918461    \n   O  Sentence 9276
918462    \n   O  Sentence 9276
       Token Tag     Sentence #
918583    \n   O  Sentence 9277
918584    \n   O  Sentence 9277
       Token Tag     Sentence #
918671    \n   O  Sentence 9278
918672    \n   O  Sentence 9278
       Token Tag     Sentence #
918779    \n   O  Sentence 9279
918780    \n   O  Sentence 9279
       Token Tag     Sentence #
919062    \n   O  Sentence 9280
919063    \n   O  Sentence 9280
       Tok

       Token Tag     Sentence #
932544    \n   O  Sentence 9428
932545    \n   O  Sentence 9428
       Token Tag     Sentence #
933415    \n   O  Sentence 9429
933416    \n   O  Sentence 9429
       Token Tag     Sentence #
933499    \n   O  Sentence 9430
933500    \n   O  Sentence 9430
       Token Tag     Sentence #
933519    \n   O  Sentence 9431
933520    \n   O  Sentence 9431
       Token Tag     Sentence #
933547    \n   O  Sentence 9432
933548    \n   O  Sentence 9432
       Token Tag     Sentence #
933700    \n   O  Sentence 9433
933701    \n   O  Sentence 9433
       Token Tag     Sentence #
933784    \n   O  Sentence 9434
933785    \n   O  Sentence 9434
       Token Tag     Sentence #
933887    \n   O  Sentence 9435
933888    \n   O  Sentence 9435
       Token Tag     Sentence #
934062    \n   O  Sentence 9436
934063    \n   O  Sentence 9436
       Token Tag     Sentence #
934203    \n   O  Sentence 9437
934204    \n   O  Sentence 9437
       Token Tag     Sentence #
934272  

952252    \n   O  Sentence 9589
       Token Tag     Sentence #
952347    \n   O  Sentence 9590
952348    \n   O  Sentence 9590
       Token Tag     Sentence #
952365    \n   O  Sentence 9591
952366    \n   O  Sentence 9591
       Token Tag     Sentence #
952384    \n   O  Sentence 9592
952385    \n   O  Sentence 9592
       Token Tag     Sentence #
952656    \n   O  Sentence 9593
952657    \n   O  Sentence 9593
       Token Tag     Sentence #
952726    \n   O  Sentence 9594
952727    \n   O  Sentence 9594
       Token Tag     Sentence #
952776    \n   O  Sentence 9595
952777    \n   O  Sentence 9595
       Token Tag     Sentence #
952855    \n   O  Sentence 9596
952856    \n   O  Sentence 9596
       Token Tag     Sentence #
952960    \n   O  Sentence 9597
952961    \n   O  Sentence 9597
       Token Tag     Sentence #
953164    \n   O  Sentence 9598
953165    \n   O  Sentence 9598
       Token Tag     Sentence #
953186    \n   O  Sentence 9599
953187    \n   O  Sentence 9599
       T

In [19]:
# Cria um array com as posições a serem retiradas.
pos = []
for i in range(len(starts)):
    pos.append(starts[i])
    pos.append(starts[i]+1)
pos[:5]

[12, 13, 24, 25, 98]

In [20]:
# Remove as linhas do dataframe e reseta os índices.
combined_csv = combined_csv.drop(pos).reset_index(drop=True)

In [21]:
# Confirma se a remoção foi bem sucedida.
combined_csv.head(15)

,Token,Tag,Sentence #
0,Ementa,INICIO_ARQ,Sentence 1
1,,O,Sentence 1
2,e,O,Sentence 1
3,,O,Sentence 1
4,Acórdão,O,Sentence 1
5,,O,Sentence 1
6,06/03/2018,O,Sentence 1
7,,O,Sentence 1
8,PRIMEIRA,O,Sentence 1
9,,O,Sentence 1


# Mover B_  com Token vazio para linha abaixo

In [22]:
# 'begins' identifica as situações onde a Tag começa com 'B_' e o Token é vazio, uma situação onde 
# o anotador começou a marcação de um espaço vazio gerando a inconsistância.

begins = combined_csv[(combined_csv['Token']==' ') & 
                      (combined_csv['Tag'].str.startswith('B_'))].index.values

In [23]:
begins[0], len (begins)

(13097, 306)

In [24]:
%%time
# Rodar apenas uma vez
for i in range(len(begins)):
    combined_csv.Tag.iloc[begins[i]+1] = combined_csv.Tag.iloc[begins[i]] #Acertar a Tag do Token para começar
                                                                          #sem espaço
    combined_csv.Tag.iloc[begins[i]] = 'O' #Marca o espaço vazio como 'O'

In [25]:
i, n = 1, 10
combined_csv.iloc[begins[i]-n+7:begins[i]+n]

,Token,Tag,Sentence #
33259,termos,O,Sentence 315
33260,,O,Sentence 315
33261,do,O,Sentence 315
33262,,O,Sentence 315
33263,art,B_Ref. Legislativa,Sentence 315
33264,.,I_Ref. Legislativa,Sentence 315
33265,,I_Ref. Legislativa,Sentence 315
33266,102,I_Ref. Legislativa,Sentence 315
33267,",",I_Ref. Legislativa,Sentence 315
33268,,I_Ref. Legislativa,Sentence 315


# Tratar caracteres especiais

In [26]:
df_teste = combined_csv.copy()

In [105]:
# Identifica os tokens que começam com '§' e são maiores que 1 pois queremos pegar os casos onde
# '§' está associado ao numero do paragrafo.
espec_carac = df_teste[(df_teste.Token.str.startswith('§')) & (df_teste.Token.str.len()>1)]
espec_carac.head()

,Token,Tag,Sentence #
5083,§§,I_Ref. Legislativa,Sentence 68
29231,§3º,B_Ref. Legislativa,Sentence 286
33309,§3º,B_Ref. Legislativa,Sentence 315
42190,§1º,I_Ref. Legislativa,Sentence 398
42237,§3º,O,Sentence 398


In [123]:
espec_carac.Token.str.len().unique()

array([2, 3])

In [124]:
i = espec_carac.index
df_teste.iloc[i[0]].Token[0]

'§'

In [125]:
df_sentenc = combined_csv[combined_csv['Sentence #']=='Sentence 68'].reset_index(drop=True)
df_sentenc

,Token,Tag,Sentence #
0,9,O,Sentence 68
1,.,O,Sentence 68
2,,O,Sentence 68
3,Como,O,Sentence 68
4,,O,Sentence 68
...,...,...,...
146,da,I_Ref. Legislativa,Sentence 68
147,,I_Ref. Legislativa,Sentence 68
148,Constituição,I_Ref. Legislativa,Sentence 68
149,:,I_Ref. Legislativa,Sentence 68


In [110]:
df_sentenc[df_sentenc.Token=='§§'].index.values

array([137])

In [111]:
df_sentenc.iloc[137-3:137+5]

,Token,Tag,Sentence #
134,220,I_Ref. Legislativa,Sentence 68
135,",",I_Ref. Legislativa,Sentence 68
136,,I_Ref. Legislativa,Sentence 68
137,§§,I_Ref. Legislativa,Sentence 68
138,,I_Ref. Legislativa,Sentence 68
139,1º,I_Ref. Legislativa,Sentence 68
140,,I_Ref. Legislativa,Sentence 68
141,e,I_Ref. Legislativa,Sentence 68


In [126]:
# Splitting o caracter '§'
list(df_sentenc.iloc[137].Token)

['§', '§']

In [66]:
df_sentenc.index.values[137]

137

In [132]:
line.Tag.startswith('I_')

True

In [139]:
line = df_sentenc.iloc[137]
splt = list(line.Token)
if line.Tag.startswith('I_') == True:
    linha = pd.DataFrame(line).transpose()
linha

,Token,Tag,Sentence #
137,§§,I_Ref. Legislativa,Sentence 68


In [114]:
df_sentenc = df_sentenc.append(linha, ignore_index=False)
df_sentenc = df_sentenc.sort_index().reset_index(drop=True)
df_sentenc.iloc[137-3:137+5]

,Token,Tag,Sentence #
134,220,I_Ref. Legislativa,Sentence 68
135,",",I_Ref. Legislativa,Sentence 68
136,,I_Ref. Legislativa,Sentence 68
137,§,I_Ref. Legislativa,Sentence 68
138,§,I_Ref. Legislativa,Sentence 68
139,,I_Ref. Legislativa,Sentence 68
140,1º,I_Ref. Legislativa,Sentence 68
141,,I_Ref. Legislativa,Sentence 68


In [113]:
# # for i in df_sentenc.index.values[137]:
# line = df_sentenc.iloc[137]
# splt = list(line.Token)
# if len(splt)==2:
#     df_sentenc.iloc[137].Token = splt[0]
#     df_sentenc.iloc[137.5].Token = splt[1]
#     df_sentenc.sort_index().reset_index(drop=True)
        
# df_sentenc[137-3:137+5]

,Token,Tag,Sentence #
134,220,I_Ref. Legislativa,Sentence 68
135,",",I_Ref. Legislativa,Sentence 68
136,,I_Ref. Legislativa,Sentence 68
137,§,I_Ref. Legislativa,Sentence 68
138,,I_Ref. Legislativa,Sentence 68
139,1º,I_Ref. Legislativa,Sentence 68
140,,I_Ref. Legislativa,Sentence 68
141,e,I_Ref. Legislativa,Sentence 68


In [85]:
pd.DataFrame({"onset": 30.0, "length": 1.3}, index=[2.5])

,onset,length
2.5,30.0,1.3


In [99]:
line

Token                          §
Tag           I_Ref. Legislativa
Sentence #           Sentence 68
Name: 137, dtype: object

In [103]:
pd.DataFrame(line).transpose()

,137.5
137,NaN


In [86]:
pd.DataFrame(line, index=[137+0.5])

,137
137.5,NaN


In [109]:
[i for i in espec_carac.Token.iloc[0]]

['§', '§']

In [129]:
espec_carac.index

Int64Index([  5083,  29231,  33309,  42190,  42237,  42326,  57473,  60988,
             65243,  65505,
            ...
            909631, 915349, 921182, 921834, 921989, 922201, 926525, 926752,
            927449, 929647],
           dtype='int64', length=108)

In [181]:
tkn = combined_csv.iloc[espec_carac.index[0]]
tkn_expand = [i for i in tkn.Token]
tkn['Sentence #']

'Sentence 68'

In [190]:
# Function to insert row in the dataframe 
def Insert_row(row_number, df, row_value): 
    start_upper = 0 # Starting value of upper half 
    end_upper = row_number  # End value of upper half 
    start_lower = row_number # Start value of lower half 
    end_lower = df.shape[0] # End value of lower half

    # Create a list of upper_half index 
    upper_half = [*range(start_upper, end_upper, 1)]
	# Create a list of lower_half index 
    lower_half = [*range(start_lower, end_lower, 1)] 
	# Increment the value of lower half by 1 
    lower_half = [x.__add__(1) for x in lower_half]

	# Combine the two lists 
    index_ = upper_half + lower_half 

	# Update the index of the dataframe 
    df.index = index_ 

	# Insert a row at the end 
    df.loc[row_number] = row_value 
	
	# Sort the index labels 
    df = df.sort_index() 

	# return the dataframe 
    return df 

In [192]:
tkn.Token[0]

'§'

In [197]:
# Let's create a row which we want to insert 
row_number = espec_carac.index[0]#2
row_value = [tkn.Token[0], tkn.Tag, tkn['Sentence #']] 

if row_number > df.index.max()+1: 
	print("Invalid row_number") 
else: 
	
	# Let's call the function and insert the row 
	# at the second position 
	df_carac = Insert_row(row_number, combined_csv, row_value) 

	# Print the updated dataframe 
	print(df_carac.iloc[row_number-3:row_number+3]) 

     Token                 Tag   Sentence #
5080   220  I_Ref. Legislativa  Sentence 68
5081     ,  I_Ref. Legislativa  Sentence 68
5082        I_Ref. Legislativa  Sentence 68
5083     §  I_Ref. Legislativa  Sentence 68
5084    §§  I_Ref. Legislativa  Sentence 68
5085        I_Ref. Legislativa  Sentence 68


In [184]:
# Baseline de como funciona
#        onset    length
# 1      2.215    1.3
# 2     23.107    1.3
# 3     41.815    1.3
# 4     61.606    1.3

# line = DataFrame({"onset": 30.0, "length": 1.3}, index=[2.5])
# df = df.append(line, ignore_index=False)
# df = df.sort_index().reset_index(drop=True)

In [173]:
espec_carac.index[0]+0.5

5083.5

In [182]:
line = pd.DataFrame({'Token': tkn_expand[0],"Tag": tkn.Tag, 'Sentence #': tkn['Sentence #']}, index=[espec_carac.index[0]+0.5])

In [183]:
combined_csv.append(line, ignore_index=False).sort_index().reset_index(drop=True).iloc[espec_carac.index[0]-3:espec_carac.index[0]+3]

,Token,Tag,Sentence #
5080,220,I_Ref. Legislativa,Sentence 68
5081,",",I_Ref. Legislativa,Sentence 68
5082,,I_Ref. Legislativa,Sentence 68
5083,§§,I_Ref. Legislativa,Sentence 68
5084,§,I_Ref. Legislativa,Sentence 68
5085,,I_Ref. Legislativa,Sentence 68


In [160]:
l = []
for i in range(len(espec_carac)):
    l.append([i for i in espec_carac.Token.iloc[i]])
df_carac = pd.DataFrame(l).stack(0).dropna().reset_index()
df_carac

,level_0,level_1,0
0,0,0,§
1,0,1,§
2,1,0,§
3,1,1,3
4,1,2,º
...,...,...,...
292,106,1,2
293,106,2,º
294,107,0,§
295,107,1,7


In [32]:
i, n = 3, 10
combined_csv.iloc[espec_carac.index.values[i]-(n-7):espec_carac.index.values[i]+n]

,Token,Tag,Sentence #
42187,B,I_Ref. Legislativa,Sentence 398
42188,",",I_Ref. Legislativa,Sentence 398
42189,,I_Ref. Legislativa,Sentence 398
42190,§1º,I_Ref. Legislativa,Sentence 398
42191,",",I_Ref. Legislativa,Sentence 398
42192,,I_Ref. Legislativa,Sentence 398
42193,do,I_Ref. Legislativa,Sentence 398
42194,,I_Ref. Legislativa,Sentence 398
42195,Código,I_Ref. Legislativa,Sentence 398
42196,,I_Ref. Legislativa,Sentence 398


In [252]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=False)
    return res

explode(espec_carac.assign(Token=espec_carac.Token.str.split('')), 'Token').iloc[:10]

,index,Sentence #,Tag,Token
0,5083,Sentence 68,I_Ref. Legislativa,
1,5083,Sentence 68,I_Ref. Legislativa,§
2,5083,Sentence 68,I_Ref. Legislativa,§
3,5083,Sentence 68,I_Ref. Legislativa,
4,29231,Sentence 286,B_Ref. Legislativa,
5,29231,Sentence 286,B_Ref. Legislativa,§
6,29231,Sentence 286,B_Ref. Legislativa,3
7,29231,Sentence 286,B_Ref. Legislativa,º
8,29231,Sentence 286,B_Ref. Legislativa,
9,33309,Sentence 315,B_Ref. Legislativa,


# Remove pontuação dos números

In [121]:
# Criando um DataFrame onde todas os tokens possuem pontuação 
df_pont = combined_csv[(combined_csv.Token.str.contains("""[.]"""))]

# Tokens de tamanho maior que 1.
df_pont[[len(df_pont.Token.iloc[i]) > 1 for i in range(len(df_pont.Token))]].head()

,Token,Tag,Sentence #
14,22.328,O,Sentence 2
781,22.328,O,Sentence 21
1061,26.05.2015,O,Sentence 31
1124,05.06.2013,O,Sentence 31
1349,"R$5.000,00",O,Sentence 32


In [65]:
# Encontra todas as linhas que tem dígito
k = []
for i in range(len(combined_csv.Token)):
    l = re.findall('[\d]', combined_csv.Token.iloc[i])
    if len(l) != 0:
        k.append(l)
# k

In [118]:
# combined_csv.Token.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# combined_csv.Token.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
combine_teste = combined_csv[combined_csv.Token.str.len()>1].index.values
combine_teste2 = combined_csv
[combine_teste2.Token.str.len()>1].Token.replace(regex=True,inplace=True,to_replace=r'\D',value='')
combine_teste2

array([     0,      4,      6, ..., 948488, 948490, 948493])

In [125]:
for i in combined_csv[combined_csv.Token.str.len()>1].index.values:
    combined_csv.iloc[i].Token.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

TypeError: replace() takes no keyword arguments

# IMPORTANTE: Esse último passo é necessário para salvar todas as alterações feitas no preprocessamento.

In [185]:
# Salvando o processamento feito nos dados
combined_csv.to_csv("preprocessados.csv",index=False,encoding='utf-8')

# Olhando para a predição de cada X

In [22]:
X_test[0][0]['word.lower()'], y_pred[0][0], y_test[0][0]

NameError: name 'X_test' is not defined

In [ ]:
# Cria dataframe com o que o modelo predisse para análise.

result = []
for i in range(len(y_test)):
    df = pd.DataFrame(zip([X_test[i][j]['word.lower()'] for j in range(len(X_test[i]))], y_test[i], y_pred[i]))
#     df = pd.DataFrame(zip(X_test[i], y_test[i], y_pred[i]))
    result.append(df)
    
result = pd.concat(result)
result.reset_index(inplace=True, drop=True)

In [ ]:
# combined_csv = pd.concat(frames)
result.to_csv("resultado.csv", index=False, encoding='utf-8')

In [ ]:
result.shape

In [ ]:
result.columns = ['X_test','y_test', 'y_pred']
result.head()

In [ ]:
result[result['y_test'] != result['y_pred']]

In [ ]:
print('Proporção de erros:',result[result.y_test != result.y_pred].shape[0] / result.shape[0])

----------------------------------------------------------------------------------------

In [ ]:
df_res = pd.read_csv('mock/resultado.csv')
df_combined = pd.read_csv('mock/combined_csv.csv')

In [ ]:
df_res.head()

In [ ]:
df_res.shape

In [ ]:
df_erros = df_res[df_res['y_test'] != df_res['y_pred']]
df_erros.head()

In [ ]:
df_erros[(df_erros['X_test']==' ') & (df_erros['y_test'].str.startswith('B_'))]
indices = df_erros[(df_erros['X_test']==' ') & (df_erros['y_test'].str.startswith('B_'))].index.values

In [ ]:
n = 10
pos = indices[1]

print('Retorna %d linhas antes e depois do ocorrido na linha %d para tag que começa com B_'%(n,pos)),df_res.iloc[pos-n:pos+n]

In [ ]:
df_res

In [ ]:
for i in indices:
    df.iloc[pos-n:pos+n]

# DUMP

In [ ]:
# combined_csv.applymap(np.isreal)
# ord(combined_csv.Token)#<128
# combined_csv.Token[:10]
ord(combined_csv.Token.iloc[0])

In [ ]:
import string
alphabet = string.ascii_letters#+string.punctuation
alphabet 

In [ ]:
regexPattern = re.compile(alphabet)

In [ ]:
combined_csv.Token = combined_csv.Token.astype('str')

In [ ]:
combined_csv.iloc[[3,5]]

In [ ]:
regexPattern.findall(combined_csv.Token.iloc[i])

In [ ]:
# combined_csv.Token[combined_csv.Token.str.contains(alphabet) == False]

# regexPattern.findall(combined_csv.Token)

l = []
for i in range(len(combined_csv.Token)):
#     l.append(regexPattern.findall(combined_csv.Token.iloc[i]).index)
    indx = regexPattern.findall(combined_csv.Token.iloc[i]).index.values
    l.append(indx)
l

In [ ]:
alphabet = string.ascii_letters
mainStr = combined_csv.Token.iloc[0]

# Create a regex pattern to match character 's'
regexPattern = re.compile(alphabet)
 
# Iterate over all the matches of regex pattern
iteratorOfMatchObs = regexPattern.finditer(mainStr)
indexPositions = []
count = 0
for matchObj in iteratorOfMatchObs:
    indexPositions.append(matchObj.start())
    count = count + 1
 
# print("Occurrence Count of character 's' : ", count)
print("Index Positions of 's' are : ", indexPositions)


In [ ]:
l = []
for i in range(len(combined_csv)):
    iteratorOfMatchObs = regexPattern.finditer(combined_csv.Token.iloc[i])
    indexPositions = []
    count = 0
    for matchObj in iteratorOfMatchObs:
        indexPositions.append(matchObj.start())
        count = count + 1
    l.append(indexPositions)

In [ ]:
l.count([])

In [ ]:
regexPattern.findall(combined_csv.Token.iloc[0]).index

In [ ]:
# combined_csv[combined_csv.Token.str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~123456789]""")]

# combined_csv[(combined_csv.Token.str.contains("""[./]"""))& (combined_csv.Token.str.len() > 1)]

In [17]:
# No meio do preprocessamento foi identificado tokens que terminam com 'adv' e estão colados ao nome do
# advogado referente ao caso.
# Tokens que terminan com 'adv'.

adv = combined_csv[combined_csv.Token.str.endswith('ADV')]#.reset_index(drop=True)
adv.head()#, adv.shape

,Token,Tag,Sentence #
46,ADV,O,Sentence 3
198,ADV,O,Sentence 9
234,ADV,O,Sentence 9
813,ADV,O,Sentence 22
965,ADV,O,Sentence 28


In [18]:
# Desconfiança que o nome do advogado está agregado à palavra 'adv'

adv[adv.Token.str.len() >3]
adv_pos = adv[adv.Token.str.len() >3].index.values

k = 2 # Vê as linhas antes e depois do k-ésimo ocorrido .
combined_csv.iloc[adv_pos[k]-3:adv_pos[k]+3] # Olhando para as linhas anteriores e posteriores o acontecimento.
# combined_csv.iloc[adv_pos[0]].Token[-3:] #Separando a parte 'adv'

,Token,Tag,Sentence #
19453,,O,Sentence 203
19454,SOFIA,B_Pessoa,Sentence 203
19455,,I_Pessoa,Sentence 203
19456,CERQUEIRAADV,O,Sentence 203
19457,.,O,Sentence 203
19458,(,O,Sentence 203


In [19]:
print("Quantidade de vezes que esse caso ocorre em todos os arquivos:",len(adv_pos))

Quantidade de vezes que esse caso ocorre em todos os arquivos: 70
